In [1]:
from IPython.display import display
from PIL import Image


In [3]:
import os
SLACK_ID = os.environ['SLACK_ID']
from slacker import Slacker
slack = Slacker(SLACK_ID)
def slack_log(message):
    try:
        slack.chat.post_message('#kent_test', message)
    except:
        print "slack connection error"
def slack_img(path):
    slack.files.upload(path,channels="#kent_test")

In [7]:
from IPython.display import clear_output
from matplotlib.pyplot import imshow
%matplotlib inline  

import time

# -*- coding:utf-8 -*-
import cv2

from boss_train import Model
# from image_show import show_image


if __name__ == '__main__':
    cap = cv2.VideoCapture(0)
    cascade_path = "/usr/local/opt/opencv/share/OpenCV/haarcascades/haarcascade_frontalface_default.xml"
    model = Model()
    model.load()
    while True:
        _, frame = cap.read()

        # グレースケール変換
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # カスケード分類器の特徴量を取得する
        cascade = cv2.CascadeClassifier(cascade_path)

        # 物体認識（顔認識）の実行
        facerect = cascade.detectMultiScale(frame_gray, scaleFactor=1.2, minNeighbors=3, minSize=(10, 10))
        #facerect = cascade.detectMultiScale(frame_gray, scaleFactor=1.01, minNeighbors=3, minSize=(3, 3))
        if len(facerect) > 0:
            print('face detected')
            color = (255, 255, 255)  # 白
            for rect in facerect:
                clear_output()
                # 検出した顔を囲む矩形の作成
                #cv2.rectangle(frame, tuple(rect[0:2]), tuple(rect[0:2] + rect[2:4]), color, thickness=2)

                x, y = rect[0:2]
                width, height = rect[2:4]
                image = frame[y - 10: y + height, x: x + width]

                result = model.predict(image)
                if result == 0:  # boss
                    print('Boss is approaching')
                    slack_log('Boss is approaching')
                    im = Image.fromarray(image)
                    im.save("./temp.jpg")
                    slack_img("./temp.jpg")
                else:
                    print('Not boss')


            time.sleep(2)
    #キャプチャを終了
    cap.release()
    cv2.destroyAllWindows()


1/1 [==============================] - 0s
[[ 0.62165397  0.378346  ]]
1/1 [==============================] - 0s
Boss is approaching


error: /tmp/opencv20150506-38415-u2kidu/opencv-2.4.11/modules/imgproc/src/color.cpp:3739: error: (-215) scn == 3 || scn == 4 in function cvtColor
